In [15]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import torch
from transformers import *
import numpy as np
import re
from langdetect import detect
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [165]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sumitsidana/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [172]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

In [180]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

In [132]:
pd.set_option('display.max_colwidth', -1)

In [23]:
import gensim
from gensim import models

In [36]:
import lda

In [6]:
val_input = pd.read_csv('s3://recsys-challenge-2020/val_input.csv')

In [7]:
languages = list(val_input.name.unique())

In [9]:
val_input.head()

,tweet_text,name
0,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko
1,방송 막하지 말라던 최민호는 본인이 막히기 시작하는데 t co wqz58zzz5l,ko
2,rt lovablebh _ 0506 백현이 오늘 심각하게 unk 던지고 가만히 서서...,ko
3,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko
4,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko


In [10]:
val_input.sort_values(by='name', inplace=True)

In [ ]:
for language in languages:
    val_input.loc[val_input.name == language]

In [12]:
en_val_input = val_input.loc[val_input.name == 'en']

In [13]:
en_val_input.head()

,tweet_text,name
4955884,rt girlsreallyrule the world s biggest politic...,en
4955905,this is true the us unk interactions with fore...,en
4955906,it s a bit breezy that eccles cake and custard...,en
4955907,yo bizzel1 they got me too we all gonna take a...,en
4955908,rt uznmaki hello t co 0i2gh3tefh,en


In [18]:
fr_val_input = val_input.loc[val_input.name == 'fr']

In [ ]:
stop_words = list(get_stop_words('fr'))         #About 900 stopwords
nltk_words = list(stopwords.words('french')) #About 150 stopwords
stop_words.extend(nltk_words)
stop_words.extend(['rt'])

In [185]:
def stop_word_removal(row):
    stopword_removed_output = [w for w in row.split(' ') if not w in stop_words]
    lemmatized_output = [stemmer.stem(w) for w in stopword_removed_output]
    return lemmatized_output

In [186]:
fr_val_input['tweet_tokens'] = fr_val_input.apply(lambda x: stop_word_removal(x.tweet_text), axis = 1)

/Users/sumitsidana/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [187]:
fr_val_input.head()

,tweet_text,name,tweet_tokens
10507192,rt _ momo58 les fléchettes aux states c unk est un autre délire t co pwtg8pezun,fr,"[_, momo58, fléchet, stat, unk, délir, co, pwtg8pezun]"
10507191,rt paulineolg moi tu me dis ça j unk te traite de mytho en te regardant droit dans les yeux faut que j unk adoucisse mon coeur un peu,fr,"[paulineolg, dis, unk, trait, mytho, regard, droit, yeux, faut, unk, adouc, coeur]"
10507190,rt _ momo58 les fléchettes aux states c unk est un autre délire t co pwtg8pezun,fr,"[_, momo58, fléchet, stat, unk, délir, co, pwtg8pezun]"
10507189,rt biboo _ r6 retrouvez moi dimanche 2 février à 19h30 dans l émission radio esix présenté par le magnifique r3siak unk je compte sur vous unk,fr,"[biboo, _, r6, retrouv, dimanch, 2, févri, 19h30, émiss, radio, esix, présent, magnif, r3siak, unk, compt, unk]"
10507188,rt nayonek _ pour le dire plus poliment que certains tu as percé grâce à ta victimisation sur les réseaux et sur tes vidéos le fait que unk,fr,"[nayonek, _, dir, plus, pol, certain, perc, grâc, victimis, réseau, vidéos, unk]"


In [195]:
from gensim import corpora
# dictionary = corpora.Dictionary(text_data)corpus = [dictionary.doc2bow(text) for text in text_data]
# import pickle
# pickle.dump(corpus, open('corpus.pkl', 'wb'))
# dictionary.save('dictionary.gensim')

In [198]:
dictionary = corpora.Dictionary(fr_val_input.tweet_tokens)
# corpus = [dictionary.doc2bow(text) for text in text_data]

SyntaxError: invalid syntax (<ipython-input-198-4e56f0a8741e>, line 1)

In [199]:
corpus = [dictionary.doc2bow(text) for text in fr_val_input.tweet_tokens]

In [200]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1),
  (7, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1)],
 [(0, 1),
  (7, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1)],
 [(7, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)],
 [(0, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1)],
 [(59, 1), (60, 1), (61, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1),
  (7, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29,

In [201]:
dictionary

In [202]:
import gensim
NUM_TOPICS = 5

In [204]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=1)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using symmetric eta at 0.2
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 405160 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.112*"unk" + 0.034*"co" + 0.013*"_" + 0.007*"jour" + 0.006*"non" + 0.005*"plus" + 0.005*"2" + 0.005*"000" + 0.005*"beaucoup" + 0.004*"vi"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.042*"fin" + 0.041*"jeu" + 0.041*"ptdr" + 0.040*"voil" + 0.032*"co" + 0.026*"unk" + 0.026*"_" + 0.011*"va" + 0.007*"quoi" + 0.007*"vi"
INFO:gens

INFO:gensim.models.ldamodel:topic #3 (0.200): 0.141*"unk" + 0.031*"co" + 0.012*"_" + 0.007*"fac" + 0.007*"plus" + 0.006*"franc" + 0.006*"0" + 0.005*"ren" + 0.005*"prêt" + 0.005*"accord"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.059*"unk" + 0.059*"co" + 0.020*"_" + 0.011*"fou" + 0.009*"rir" + 0.008*"plus" + 0.007*"premi" + 0.007*"an" + 0.006*"amien" + 0.006*"switch"
INFO:gensim.models.ldamodel:topic diff=0.116302, rho=0.377964
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #16000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.093*"unk" + 0.037*"co" + 0.028*"_" + 0.013*"voilà" + 0.009*"jour" + 0.007*"jok" + 0.007*"benzem" + 0.006*"bien" + 0.006*"non" + 0.006*"rap"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.051*"_" + 0.042*"co" + 0.029*"unk" + 0.012*"va" + 0.008*"demain" + 0.007*"lagrosserigolad" + 0.006*"coronavirus" + 0.006*"camilo" + 0.006*"soir" + 0.005*"don"


INFO:gensim.models.ldamodel:topic #2 (0.200): 0.186*"unk" + 0.038*"co" + 0.033*"boob" + 0.021*"quoi" + 0.018*"tweet" + 0.015*"featuring" + 0.015*"v" + 0.014*"_" + 0.013*"mist" + 0.013*"ijcmavnqzy"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.138*"unk" + 0.021*"co" + 0.013*"officiel" + 0.012*"_" + 0.006*"plus" + 0.006*"prêt" + 0.006*"compt" + 0.005*"0" + 0.005*"02" + 0.005*"jusqu"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.090*"unk" + 0.053*"co" + 0.016*"_" + 0.009*"an" + 0.008*"17" + 0.008*"rmcity" + 0.007*"el" + 0.007*"derb" + 0.007*"halamadrid" + 0.007*"voilà"
INFO:gensim.models.ldamodel:topic diff=0.079581, rho=0.267261
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #30000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.104*"unk" + 0.031*"co" + 0.027*"_" + 0.009*"vrai" + 0.008*"jour" + 0.006*"non" + 0.006*"derni" + 0.005*"aim" + 0.005*"real" + 0.005*"30"
INFO:ge

INFO:gensim.models.ldamodel:topic #1 (0.200): 0.043*"_" + 0.036*"unk" + 0.032*"co" + 0.017*"va" + 0.009*"nouvel" + 0.008*"demain" + 0.008*"don" + 0.007*"merc" + 0.007*"lagrosserigolad" + 0.007*"mercato"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.206*"unk" + 0.046*"boob" + 0.038*"quoi" + 0.034*"tweet" + 0.028*"wesh" + 0.028*"cousin" + 0.028*"cru" + 0.022*"co" + 0.020*"terrifi" + 0.010*"_"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.122*"unk" + 0.021*"co" + 0.012*"_" + 0.007*"sort" + 0.007*"plus" + 0.006*"1" + 0.006*"temp" + 0.005*"import" + 0.005*"quelqu" + 0.005*"rappel"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.087*"unk" + 0.065*"co" + 0.016*"_" + 0.012*"an" + 0.008*"premi" + 0.007*"officiel" + 0.007*"plus" + 0.005*"prêt" + 0.005*"fin" + 0.005*"défenseur"
INFO:gensim.models.ldamodel:topic diff=0.090640, rho=0.218218
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #44000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 4051

INFO:gensim.models.ldamodel:topic #1 (0.200): 0.038*"_" + 0.032*"unk" + 0.032*"co" + 0.013*"va" + 0.007*"plus" + 0.006*"don" + 0.006*"nouvel" + 0.006*"bon" + 0.005*"tabl" + 0.005*"soir"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.181*"unk" + 0.026*"co" + 0.021*"quoi" + 0.017*"boob" + 0.015*"tweet" + 0.012*"_" + 0.009*"wesh" + 0.009*"cru" + 0.009*"cousin" + 0.008*"moment"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.116*"unk" + 0.021*"co" + 0.014*"_" + 0.008*"plus" + 0.006*"1" + 0.005*"2" + 0.005*"2020" + 0.005*"aujourd" + 0.005*"hui" + 0.005*"brex"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.078*"unk" + 0.078*"co" + 0.014*"_" + 0.010*"an" + 0.010*"match" + 0.008*"homm" + 0.007*"gt" + 0.006*"animal" + 0.006*"premi" + 0.006*"paris"
INFO:gensim.models.ldamodel:topic diff=0.106402, rho=0.188982
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #58000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models

...

INFO:gensim.models.ldamodel:topic #1 (0.200): 0.043*"unk" + 0.041*"co" + 0.026*"_" + 0.013*"va" + 0.011*"pasteur" + 0.010*"plus" + 0.009*"coronavirus" + 0.007*"demain" + 0.007*"gar" + 0.007*"soir"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.150*"unk" + 0.047*"co" + 0.043*"_" + 0.012*"quoi" + 0.008*"réuss" + 0.007*"bojack" + 0.007*"horseman" + 0.007*"rwrjfirzfk" + 0.007*"aim" + 0.006*"anim"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.121*"unk" + 0.016*"co" + 0.010*"plus" + 0.009*"_" + 0.008*"temp" + 0.005*"termin" + 0.005*"seul" + 0.005*"jam" + 0.005*"femm" + 0.005*"rest"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.082*"co" + 0.049*"unk" + 0.021*"_" + 0.010*"épisod" + 0.008*"clip" + 0.007*"gt" + 0.006*"officiel" + 0.005*"premi" + 0.005*"chois" + 0.005*"conseil"
INFO:gensim.models.ldamodel:topic diff=0.058414, rho=0.076029
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #348000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 40

INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.089*"unk" + 0.041*"co" + 0.012*"_" + 0.009*"jour" + 0.007*"derni" + 0.006*"franc" + 0.006*"2" + 0.005*"3" + 0.005*"an" + 0.005*"1"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.044*"unk" + 0.042*"co" + 0.026*"_" + 0.014*"va" + 0.010*"plus" + 0.007*"coronavirus" + 0.007*"soir" + 0.007*"gros" + 0.007*"pasteur" + 0.007*"demain"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.154*"unk" + 0.043*"co" + 0.039*"_" + 0.016*"anim" + 0.011*"quoi" + 0.010*"aim" + 0.010*"sembl" + 0.007*"story" + 0.006*"stat" + 0.005*"réuss"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.120*"unk" + 0.013*"co" + 0.011*"voir" + 0.010*"attend" + 0.009*"plus" + 0.009*"croir" + 0.009*"fer" + 0.009*"peux" + 0.008*"ball" + 0.008*"possibl"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.087*"co" + 0.056*"unk" + 0.019*"_" + 0.015*"clip" + 0.014*"seb" + 0.013*"one" + 0.013*"piec"

INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #374000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.089*"unk" + 0.037*"co" + 0.012*"_" + 0.008*"jour" + 0.007*"bien" + 0.007*"derni" + 0.006*"anné" + 0.006*"2" + 0.006*"an" + 0.005*"franc"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.045*"unk" + 0.037*"co" + 0.029*"_" + 0.015*"va" + 0.011*"plus" + 0.008*"soir" + 0.006*"demain" + 0.006*"coronavirus" + 0.006*"gros" + 0.005*"occas"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.160*"unk" + 0.038*"co" + 0.037*"_" + 0.015*"quoi" + 0.010*"anim" + 0.009*"aim" + 0.008*"tweet" + 0.008*"sembl" + 0.008*"boob" + 0.006*"aller"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.123*"unk" + 0.014*"co" + 0.009*"plus" + 0.009*"_" + 0.008*"voir" + 0.006*"attend" + 0.006*"croir" + 0.006*"peux" + 0.006*"fer" + 0.006*"vrai"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.082*"co" + 0.056*"unk

INFO:gensim.models.ldamodel:topic diff=0.052016, rho=0.071982
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #388000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.091*"unk" + 0.035*"co" + 0.014*"_" + 0.008*"jour" + 0.006*"an" + 0.006*"bien" + 0.006*"2" + 0.006*"2020" + 0.006*"derni" + 0.005*"3"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.049*"unk" + 0.037*"co" + 0.030*"_" + 0.015*"va" + 0.011*"plus" + 0.007*"soir" + 0.007*"bon" + 0.007*"demain" + 0.006*"1er" + 0.005*"merc"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.159*"unk" + 0.038*"co" + 0.036*"_" + 0.019*"quoi" + 0.015*"boob" + 0.013*"tweet" + 0.008*"wesh" + 0.008*"cru" + 0.007*"cousin" + 0.007*"aller"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.128*"unk" + 0.015*"co" + 0.010*"plus" + 0.009*"_" + 0.006*"voir" + 0.006*"jam" + 0.005*"officiel" + 0.005*"vrai" + 0.005*"fin" + 0.005*"peux"
INFO:gensim.models.l

INFO:gensim.models.ldamodel:topic diff=0.048882, rho=0.070711
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #402000/405160
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 405160 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.093*"unk" + 0.035*"co" + 0.014*"_" + 0.011*"2" + 0.009*"3" + 0.008*"jour" + 0.007*"1" + 0.007*"derby" + 0.006*"derni" + 0.006*"bien"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.055*"unk" + 0.037*"co" + 0.032*"_" + 0.016*"va" + 0.011*"plus" + 0.008*"demain" + 0.007*"soir" + 0.006*"bon" + 0.006*"merc" + 0.005*"1er"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.159*"unk" + 0.037*"_" + 0.036*"co" + 0.020*"quoi" + 0.019*"boob" + 0.013*"tweet" + 0.009*"wesh" + 0.008*"cru" + 0.008*"cousin" + 0.007*"v"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.129*"unk" + 0.016*"co" + 0.010*"_" + 0.010*"ren" + 0.009*"plus" + 0.008*"nant" + 0.008*"srfcfcn" + 0.007*"voir" + 0.007*"2" + 0.005*"jam"
INFO:gensim.models.ldamodel:

In [206]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

INFO:gensim.models.ldamodel:topic #0 (0.200): 0.091*"unk" + 0.034*"co" + 0.014*"_" + 0.010*"2" + 0.008*"3" + 0.007*"jour" + 0.006*"1" + 0.006*"derby" + 0.006*"an" + 0.006*"bien"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.054*"unk" + 0.037*"co" + 0.031*"_" + 0.016*"va" + 0.011*"plus" + 0.007*"demain" + 0.007*"soir" + 0.006*"merc" + 0.006*"bon" + 0.005*"laiss"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.158*"unk" + 0.038*"_" + 0.035*"co" + 0.019*"quoi" + 0.019*"boob" + 0.013*"tweet" + 0.009*"wesh" + 0.008*"cru" + 0.008*"cousin" + 0.008*"v"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.128*"unk" + 0.016*"co" + 0.010*"_" + 0.009*"plus" + 0.008*"ren" + 0.007*"nant" + 0.007*"mec" + 0.007*"srfcfcn" + 0.006*"2" + 0.006*"voir"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.074*"co" + 0.053*"unk" + 0.020*"_" + 0.008*"gt" + 0.005*"premi" + 0.005*"match" + 0.005*"annonc" + 0.005*"condamn" + 0.005*"min" + 0.005*"clip"


(0, '0.091*"unk" + 0.034*"co" + 0.014*"_" + 0.010*"2" + 0.008*"3" + 0.007*"jour" + 0.006*"1" + 0.006*"derby" + 0.006*"an" + 0.006*"bien"')
(1, '0.054*"unk" + 0.037*"co" + 0.031*"_" + 0.016*"va" + 0.011*"plus" + 0.007*"demain" + 0.007*"soir" + 0.006*"merc" + 0.006*"bon" + 0.005*"laiss"')
(2, '0.158*"unk" + 0.038*"_" + 0.035*"co" + 0.019*"quoi" + 0.019*"boob" + 0.013*"tweet" + 0.009*"wesh" + 0.008*"cru" + 0.008*"cousin" + 0.008*"v"')
(3, '0.128*"unk" + 0.016*"co" + 0.010*"_" + 0.009*"plus" + 0.008*"ren" + 0.007*"nant" + 0.007*"mec" + 0.007*"srfcfcn" + 0.006*"2" + 0.006*"voir"')
(4, '0.074*"co" + 0.053*"unk" + 0.020*"_" + 0.008*"gt" + 0.005*"premi" + 0.005*"match" + 0.005*"annonc" + 0.005*"condamn" + 0.005*"min" + 0.005*"clip"')


In [209]:
ldamodel.get_document_topics(corpus[89])

[(0, 0.44784737),
 (1, 0.04000048),
 (2, 0.23957641),
 (3, 0.040000416),
 (4, 0.23257533)]

In [213]:
ldamodel[corpus[5]]

[(0, 0.022307329),
 (1, 0.022272337),
 (2, 0.910806),
 (3, 0.022342406),
 (4, 0.022271954)]

In [193]:
univectorizer = CountVectorizer(analyzer = "word", min_df = 0.0, ngram_range = (1,1), strip_accents= None, tokenizer = None)

In [194]:
unicorpus = univectorizer.fit_transform(fr_val_input["tweet_tokens"])

AttributeError: 'list' object has no attribute 'lower'

In [160]:
unigrams = univectorizer.get_feature_names()

In [161]:
unigrams

['00',
 '000',
 '01',
 '02',
 '10',
 '100',
 '1070',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17h',
 '18',
 '18h',
 '19',
 '1er',
 '1jnat7v6lt',
 '1ère',
 '20',
 '200',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '22',
 '23',
 '24',
 '25',
 '28',
 '29',
 '2emfoorogb',
 '2h',
 '2ème',
 '30',
 '31',
 '34',
 '3500',
 '40',
 '400',
 '42',
 '50',
 '500',
 '54',
 '60',
 '66',
 '70',
 '80',
 '90',
 '99',
 'abonnement',
 'abonnés',
 'abord',
 'abri',
 'absolument',
 'accord',
 'achat',
 'acheter',
 'acheté',
 'action',
 'activité',
 'actuellement',
 'adama',
 'adore',
 'affa',
 'affaire',
 'affaires',
 'afin',
 'afp',
 'afrique',
 'agit',
 'ah',
 'ai',
 'aide',
 'aider',
 'ailleurs',
 'aime',
 'aiment',
 'aimer',
 'aimerais',
 'aimes',
 'aimez',
 'aimé',
 'ainsi',
 'air',
 'ait',
 'al',
 'alarme',
 'album',
 'albums',
 'alcool',
 'alerte',
 'all',
 'allah',
 'aller',
 'allez',
 'alors',
 'ameeel',
 'ami',
 'amis',
 'amour',
 'amoureux',
 'amp',
 'an',
 'ancien',
 'and',
 'an

In [162]:
print(unicorpus[0,:])

  (0, 1351)	1
  (0, 837)	1
  (0, 172)	1
  (0, 1592)	1
  (0, 568)	1
  (0, 1583)	1
  (0, 170)	1
  (0, 344)	1


In [163]:
unicorpus.shape

(405160, 1717)

In [157]:
unigrams[10204]

'states'

In [41]:
lda_model = lda.LDA(n_topics = 10)

In [42]:
lda_model.fit(unicorpus)

INFO:lda:n_documents: 405160
INFO:lda:vocab_size: 1717
INFO:lda:n_words: 5508855
INFO:lda:n_topics: 10
INFO:lda:n_iter: 2000
/Users/sumitsidana/anaconda3/lib/python3.6/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -49198268
INFO:lda:<10> log likelihood: -37314194
INFO:lda:<20> log likelihood: -35262390
INFO:lda:<30> log likelihood: -34408764
INFO:lda:<40> log likelihood: -34008481
INFO:lda:<50> log likelihood: -33807183
INFO:lda:<60> log likelihood: -33716505
INFO:lda:<70> log likelihood: -33664773
INFO:lda:<80> log likelihood: -33629824
INFO:lda:<90> log likelihood: -33600894
INFO:lda:<100> log likelihood: -33578010
INFO:lda:<110> log likelihood: -33563485
INFO:lda:<120> log likelihood: -33545894
INFO:lda:<130> log likelihood: -33538811
INFO

INFO:lda:<1870> log likelihood: -33384936
INFO:lda:<1880> log likelihood: -33386229
INFO:lda:<1890> log likelihood: -33389440
INFO:lda:<1900> log likelihood: -33383866
INFO:lda:<1910> log likelihood: -33377075
INFO:lda:<1920> log likelihood: -33363686
INFO:lda:<1930> log likelihood: -33371542
INFO:lda:<1940> log likelihood: -33375270
INFO:lda:<1950> log likelihood: -33369721
INFO:lda:<1960> log likelihood: -33371110
INFO:lda:<1970> log likelihood: -33369066
INFO:lda:<1980> log likelihood: -33366423
INFO:lda:<1990> log likelihood: -33363974
INFO:lda:<1999> log likelihood: -33367657


In [ ]:
lda_model.

In [43]:
doc_topic = lda_model.doc_topic_

In [60]:
len(doc_topic)

405160

In [56]:
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_word_indexes = [np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_words = np.array(unigrams)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    print(topic_word_indexes)

Topic 0: les rt tu que unk et on de
[array([   0, 1076, 1074, ..., 1572, 1351,  837])]
Topic 1: unk vous les rt de des pas qui
[array([   0,  953,  952, ...,  837, 1657, 1592])]
Topic 2: unk co de la le pour sur rt
[array([   0, 1181, 1180, ...,  430,  344, 1592])]
Topic 3: de la unk co le et les des
[array([   0,  499, 1442, ..., 1592,  820,  430])]
Topic 4: unk est il rt le on qu pas
[array([1716,  735,  736, ...,  745,  568, 1592])]
Topic 5: unk co est rt de le en hui
[array([   0, 1081, 1079, ...,  568,  344, 1592])]
Topic 6: unk de co le la pour en du
[array([ 858, 1017, 1015, ...,  344,  430, 1592])]
Topic 7: unk rt je quand tu que pas est
[array([   0, 1078, 1076, ...,  782, 1351, 1592])]
Topic 8: je unk rt de ai que la un
[array([   0,  949,  948, ..., 1351, 1592,  782])]
Topic 9: unk co rt est il de un la
[array([   0, 1080, 1079, ..., 1351,  344, 1592])]


In [58]:
print(topic_word.shape)

(10, 1717)


In [46]:
len(fr_val_input)

405160